## 1. Simple Elastix

### Registration

In [15]:
# First import is currently necessary to run ITKElastix on MacOs
from itk import itkElastixRegistrationMethodPython
import itk
from itkwidgets import compare, checkerboard
import numpy as np

**Image registration** finds the *spatial transformation that aligns images in the presence of noise*.

In image registration, we typically identify the two images as the fixed and moving image. Our goal is to find the spatial transformation that makes the moving image align with the fixed image.

First, let's load our **fixed image** and the image we will align to our fixed image, the **moving image**.

In [16]:
# Load images with itk floats (itk.F). Necessary for elastix
fixed_image = itk.imread('data/CT_2D_head_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_2D_head_moving.mha', itk.F)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


For now we will use a default parametermap of elastix, for more info about parametermaps, see [example2](ITK_Example02_CustomOrMultipleParameterMaps.ipynb#section_id2)

In [17]:
parameter_object = itk.ParameterObject.New()
default_rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
parameter_object.AddParameterMap(default_rigid_parameter_map)

Registration can either be done in one line with the registration function...

In [18]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    log_to_console=False)

.. or by initiating an elastix image filter object. The result of these to methods should only differ due to the stochastic nature of elastix. 

The object oriented method below can be used when more explicit function calls are preferred.

In [19]:
# Load Elastix Image Filter Object
elastix_object = itk.ElastixRegistrationMethod.New()
elastix_object.SetFixedImage(fixed_image)
elastix_object.SetMovingImage(moving_image)
elastix_object.SetParameterObject(parameter_object)

# Set additional options
elastix_object.SetLogToConsole(False)

# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()

# Results of Registration
result_image = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()

In [20]:
# Save image with itk
itk.imwrite(result_image,'exampleoutput/result_image.mha')

The output of the elastix algorithm is the registered (transformed) version of the moving image. The parameters of this transformation can also be obtained after registation.

### Visualization
The results of the image registration can be visualized with widgets from the itkwidget library such as the checkerboard and compare widgets.

In [7]:
checkerboard(fixed_image, result_image)

In [21]:
compare(fixed_image, result_image, link_cmap=True)

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=True, description='cmap'), Checkbox(va…

In [11]:
# import napari

In [12]:
# Cast result images to numpy arrays for Napari Viewer
# fixed_image_np = np.asarray(fixed_image).astype(np.float32)
# moving_image_np = np.asarray(moving_image).astype(np.float32)
# result_image_np = np.asarray(result_image).astype(np.float32)

In [1]:
# with napari.gui_qt():
#     viewer = napari.Viewer()
#     viewer.add_image(fixed_image_np)
#     viewer.add_image(moving_image_np)
#     viewer.add_image(result_image_np)
